In [1]:

URL = 'jdbc:oracle:thin:system/"123456"@localhost:1521/XE'
# USER: sys and system have more rights -> only with serivce name XE
SERVICE_NAME = "XE"
USER = "system"
PASSWORD = "123456"
DBTABLE = "system.test_table"
DRIVER = "oracle.jdbc.OracleDriver"
BUILD_PATH = "/workspaces/testing_pyspark/build/*"

In [8]:
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext

conf = SparkConf() \
        .setAppName("SarkTest") \
        .set("spark.driver.host", "127.0.0.1") \
        .set("spark.driver.extraClassPath", BUILD_PATH) \
        .set("spark.executor.extraClassPath", BUILD_PATH)

spark = SparkSession.builder.config(conf=conf).getOrCreate()

sc = spark.sparkContext
sql_context = SQLContext(sparkContext=sc, sparkSession=spark)
sc.setLogLevel("WARN")

In [10]:
jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", URL) \
    .option("dbtable", DBTABLE) \
    .option("driver", DRIVER) \
    .option("oracle.jdbc.mapDateToTimestamp", "false") \
    .load()

jdbcDF.count()

3

In [11]:
jdbcDF.show(truncate=False)

+---+-------------------+--------------------------+
|ID |TODAY              |RIGHT_NOW                 |
+---+-------------------+--------------------------+
|1  |2024-08-04 03:37:59|2024-08-04 03:37:59.403974|
|2  |2024-08-04 03:37:59|2024-08-04 03:37:59.406956|
|3  |2024-08-04 03:37:59|2024-08-04 03:37:59.411245|
+---+-------------------+--------------------------+



In [12]:
jdbcDF.write.parquet("build/test.parquet", mode="overwrite")
output_parquet = spark.read.parquet("build/test.parquet")
output_parquet.show(truncate=False)

+---+-------------------+--------------------------+
|ID |TODAY              |RIGHT_NOW                 |
+---+-------------------+--------------------------+
|1  |2024-08-04 03:37:59|2024-08-04 03:37:59.403974|
|2  |2024-08-04 03:37:59|2024-08-04 03:37:59.406956|
|3  |2024-08-04 03:37:59|2024-08-04 03:37:59.411245|
+---+-------------------+--------------------------+



In [13]:
print(spark.conf.get("spark.sql.session.timeZone"))

Etc/UTC
